In [ ]:
pip install llama-index llama-index-llms-ollama llama-index-embeddings-ollama

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
import os

In [2]:
Settings.llm = Ollama(model="llama3.1", request_timeout=120.0) 

# โมเดล Embedding สำหรับแปลงข้อความเป็น Vector (สำคัญมากสำหรับ RAG!)
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text") 

# ตั้งค่าขนาด Chunk (แนะนำสำหรับ RAG เบื้องต้น)
Settings.chunk_size = 512
Settings.chunk_overlap = 20

print("--- การกำหนดค่า Ollama และ Embedding เสร็จสมบูรณ์ ---")

# --- 2. โหลดข้อมูลและสร้าง Index ---
# โหลดไฟล์ทั้งหมดจากโฟลเดอร์ ./data
# ตรวจสอบให้แน่ใจว่าคุณมีไฟล์ในโฟลเดอร์ 'data' ก่อนรัน
if not os.path.exists("./data"):
    print("กรุณาสร้างโฟลเดอร์ 'data' และใส่เอกสารเข้าไป")
else:
    documents = SimpleDirectoryReader(input_dir="./data").load_data()
    print(f"โหลดเอกสารทั้งหมด {len(documents)} ไฟล์จาก ./data")

    # สร้าง Vector Index
    # นี่คือขั้นตอนที่นำเอกสารไปแบ่งส่วน (Chunk) แปลงเป็น Vector (Embed) 
    # และจัดเก็บ (Store) ใน Index (โดยค่าเริ่มต้นจะใช้ SimpleVectorStore ในหน่วยความจำ)
    index = VectorStoreIndex.from_documents(documents)
    print("--- สร้าง Vector Index เสร็จสมบูรณ์ ---")

    # --- 3. สร้าง Query Engine และเริ่มถามคำถาม ---
    # Query Engine จะเป็นตัวจัดการขั้นตอนการค้นหาและสร้างคำตอบ
    query_engine = index.as_query_engine(
        similarity_top_k=3 # ให้ค้นหา 3 ชิ้นส่วนของเอกสารที่เกี่ยวข้องที่สุด
    )

    # ทดสอบถามคำถาม
    user_question = "สรุปเนื้อหาหลักเกี่ยวกับโครงการนี้ให้หน่อย"
    
    print("\n\n--- เริ่มตอบคำถาม ---")
    print(f"คำถาม: {user_question}")
    
    response = query_engine.query(user_question)

    # --- 4. แสดงผลลัพธ์ ---
    print("\n--- คำตอบที่ได้จาก RAG ---")
    print(str(response))
    
    # แสดงแหล่งข้อมูลที่ใช้ (Source)
    print("\n--- แหล่งข้อมูลที่ใช้ (Source Chunks) ---")
    for node in response.source_nodes:
        print(f"- **ไฟล์:** {node.metadata.get('file_path', 'N/A')}")
        print(f"  **เนื้อหา:** {node.text[:100]}...") # แสดงตัวอย่าง 100 ตัวอักษรแรก

--- การกำหนดค่า Ollama และ Embedding เสร็จสมบูรณ์ ---
โหลดเอกสารทั้งหมด 2 ไฟล์จาก ./data


2025-09-27 19:52:25,296 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,339 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,478 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,517 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,576 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,613 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,655 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,694 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,736 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-27 19:52:25,776 - INFO - HTTP Request: POST http://localhost:1143

--- สร้าง Vector Index เสร็จสมบูรณ์ ---


2025-09-27 19:52:28,459 - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
2025-09-27 19:52:28,515 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"




--- เริ่มตอบคำถาม ---
คำถาม: สรุปเนื้อหาหลักเกี่ยวกับโครงการนี้ให้หน่อย


2025-09-27 19:52:34,442 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 500 Internal Server Error"


ResponseError: model requires more system memory (19.7 GiB) than is available (15.9 GiB) (status code: 500)